In [1]:
#Import Packages
from selenium import webdriver
import time
import pandas as pd
import os
from bs4 import BeautifulSoup

In [2]:
#Import Packages

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [3]:
url = "https://www.foundit.in/srp/results?query=%22Software+Developer%22%2C%22Data+Analyst%22%2C%22QA%22%2C%22Quality+Control+Engineer%22%2C%22Business+Development+Manager%22%2C%22Business+Development+Executive%22%2C%22Customer+Success+Manager%22&experienceRanges=0%7E0&experience=0&searchId=06c2e8af-42c8-48b9-9ca4-0c36d5ba9381"

In [41]:
driver = webdriver.Chrome() # Initialize Chrome WebDriver
driver.implicitly_wait(2) # Wait for up to 2 seconds for elements to appear
driver.get(url) # Open the URL

In [42]:
d = {"Company_name":[], "Role":[],  "Skills":[], "Job_description":[],"Salary":[], "Experience":[], "Location":[]} # Foundit has no Application link
print(d)

{'Company_name': [], 'Role': [], 'Skills': [], 'Job_description': [], 'Salary': [], 'Experience': [], 'Location': []}


In [43]:
def get_skills(soup):
    try:
        div_skill = soup.find('div', class_='skillSet')
        # Find all <span> elements within the <div> element
        span_elements = div_skill.find_all('div', class_='skillDesc')
        if span_elements:
            # Extract the text content from each <span> element
            skills = [span.text.strip() for span in span_elements]

    except AttributeError:
        skills = " "
    return skills
def get_location(soup):
    try:
        result_card = soup.find('div', class_='srpJdContainer')

        container = result_card.find('div', class_='jobHighlights')

        details_divs = container.find_all('div', class_='details')
        # Extract details and append to the list
        location = details_divs[0].get_text(strip=True)

    except AttributeError:
        location = " "
    return location

def get_exp(soup):
    try:
        result_card = soup.find('div', class_='srpJdContainer')

        container = result_card.find('div', class_='jobHighlights')

        details_divs = container.find_all('div', class_='details')
        # Extract details and append to the list
        experience = details_divs[1].get_text(strip=True)

    except AttributeError:
        experience = " "
    return experience

def get_salary(soup):
    try:
        salary_range='null'
        result_card = soup.find('div', class_='srpJdContainer')

        container = result_card.find('div', class_='jobHighlights')

        details_divs = container.find_all('div', class_='details')
        if len(details_divs) > 2:
            salary_range = details_divs[2].get_text(strip=True)

    except AttributeError:
        salary_range = " "
    return salary_range


In [44]:
def get_info(divs):
    for div in divs:
        driver.execute_script("arguments[0].click();", div)
        time.sleep(2)  # wait for the data to load
        # Wait until the page is fully loaded
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'jobDescription')))
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        time.sleep(2)
        job_title_element = soup.find('div', class_='jdTitle')
        if job_title_element:
            job_title_text = job_title_element.span.text.strip()
            print(job_title_text)
            d["Role"].append(job_title_text)
        else:
            print("Job title not found for this entry.")
            d["Role"].append(None) 
        company_div = soup.find('div', class_='jdCompanyName')
        # Extract and print the text content inside the p tag
        if company_div:
            p_tag = company_div.find('p')
            if p_tag:
                company_name = p_tag.get_text(strip=True)
                d['Company_name'].append(company_name)
        job_description_div = soup.find('div', class_='jobDescription')

        # Extract and print all the information inside the div
        if job_description_div:
            jd = job_description_div.get_text(strip=True)
            d["Job_description"].append(jd)
        # div_skill = soup.find('div', class_='skillSet')
        # # Find all <span> elements within the <div> element
        # span_elements = div_skill.find_all('div', class_='skillDesc')
        # if span_elements:
        #     # Extract the text content from each <span> element
        #     skills = [span.text.strip() for span in span_elements]
        d["Skills"].append(get_skills(soup))
    
        # salary_range='null'
        # result_card = soup.find('div', class_='srpJdContainer')

        # container = result_card.find('div', class_='jobHighlights')

        # details_divs = container.find_all('div', class_='details')
        # # Extract details and append to the list
        # location = details_divs[0].get_text(strip=True)
        # experience = details_divs[1].get_text(strip=True)
        # if len(details_divs) > 2:
        #     salary_range = details_divs[2].get_text(strip=True)
        d["Location"].append(get_location(soup))
        d["Experience"].append(get_exp(soup))
        d["Salary"].append(get_salary(soup))
    

In [45]:
for i in range(2): # Adjust the range as requires
    # divs = driver.find_elements(By.CSS_SELECTOR, "div.srpResultCardContainer > div")
    # get_info(divs)
    # div_elements = driver.find_element(By.CSS_SELECTOR, "div.arrow.arrow-right")
    # div_elements.click()
    # Find all div elements with class "srpResultCardContainer > div" again
    divs = driver.find_elements(By.CSS_SELECTOR, "div.srpResultCardContainer > div")
    get_info(divs)
    div_elements = driver.find_element(By.CSS_SELECTOR, "div.arrow.arrow-right")
    div_elements.click()
    driver.refresh(); 


Senior Software Developer, Software Development
Senior Data Analyst
Manager  Senior Data Analyst GSCs
Reference Data Analyst
Software Developer
Senior Software Developer
Reference Data Analyst
Investiagtions Optimisation Data Analyst GSC
Reference Data Analyst
Senior Software Developer
Senior Software Developer
Reference Data Analyst
OSM Software Developer
Data Analyst, Tableau
Senior Software Developer
Software Developer 3
Principal Software Developer
Business Development Manager
Software Developer 3
Lead Data Analyst
Software Developer 3
Senior Software Developer
Software Developer
Principal Software Developer
DATA ANALYST INTERNSHIP AT PUNE
Senior Software Developer, Software Development
Principal Software Developer
Data Analyst | Business Analyst | Fresher IT Job - Fresher Can Apply  Salary 45K
Senior Software Developer
Principal Software Developer


In [32]:
import csv

In [47]:
df = pd.DataFrame.from_dict(d)

  # Save DataFrame to CSV file
csv_file_path = "job_data_foundit.csv"
df.to_csv(csv_file_path, index=False)

In [48]:
print(d)

{'Company_name': ['S&P Global Market Intelligence', 'Oracle', 'HSBC', 'Deutsche Bank', 'Barclays', 'Oracle', 'Deutsche Bank', 'HSBC', 'Deutsche Bank', 'Oracle', 'Oracle', 'Deutsche Bank', 'Oracle', 'NTT Data', 'Oracle', 'Oracle', 'Oracle', 'Company name confidential', 'Oracle', 'S&P Global Market Intelligence', 'Oracle', 'Oracle', 'Railworld India Private Limited', 'Oracle', 'Maxgen Technologies Private Limited', 'S&P Global Market Intelligence', 'Oracle', 'Structured Learning Assistance Consultants India Private Limited', 'Oracle', 'Oracle'], 'Role': ['Senior Software Developer, Software Development', 'Senior Data Analyst', 'Manager  Senior Data Analyst GSCs', 'Reference Data Analyst', 'Software Developer', 'Senior Software Developer', 'Reference Data Analyst', 'Investiagtions Optimisation Data Analyst GSC', 'Reference Data Analyst', 'Senior Software Developer', 'Senior Software Developer', 'Reference Data Analyst', 'OSM Software Developer', 'Data Analyst, Tableau', 'Senior Software D